In [4]:
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pandas as pd
import sklearn
import os
import sys
import time
import tensorflow as tf
from tensorflow import keras

print (tf.__version__)
print (sys.version_info)
for module in  mpl, np, pd,  tf, keras:
    print (module.__name__, module.__version__)

2.0.0-beta1
sys.version_info(major=3, minor=6, micro=8, releaselevel='final', serial=0)
matplotlib 3.1.1
numpy 1.16.4
pandas 0.24.2
tensorflow 2.0.0-beta1
tensorflow.python.keras.api._v2.keras 2.2.4-tf


In [5]:
train_file = "./data/titanic/train.csv"
eval_file = "./data/titanic/eval.csv"

train_df = pd.read_csv(train_file)
eval_df = pd.read_csv(eval_file)

# print (train_df.head())
# print (eval_df.head())

In [6]:
y_train = train_df.pop('survived')
y_eval = eval_df.pop('survived')

# print (train_df.head())
# print (eval_df.head())
# print (y_train.head())
# print (y_eval.head())

In [7]:
# train_df.describe()

In [9]:
## feature columns：本质是规则
# 离散特征和连续特征

categorical_columns = ['sex','n_siblings_spouses','parch', 'class',
                     'deck', 'embark_town', 'alone']
numeric_columns = ['age', 'fare']

# feature_columns本质算是一个映射规则，定义了处理特征的方式。
feature_columns =[]

for categorical_column in categorical_columns:
    vocab = train_df[categorical_column].unique() #获取不重复的值
    print (categorical_column,vocab)
    
    #indicator_column用于处理成one_hot编码
    feature_columns.append(
        tf.feature_column.indicator_column(
            tf.feature_column.categorical_column_with_vocabulary_list(
                categorical_column, vocab)))
    
for numeric_column in numeric_columns:
    feature_columns.append(
        tf.feature_column.numeric_column(
            numeric_column,dtype=tf.float32))


sex ['male' 'female']
n_siblings_spouses [1 0 3 4 2 5 8]
parch [0 1 2 5 3 4]
class ['Third' 'First' 'Second']
deck ['unknown' 'C' 'G' 'A' 'B' 'D' 'F' 'E']
embark_town ['Southampton' 'Cherbourg' 'Queenstown' 'unknown']
alone ['n' 'y']


In [10]:
#tf.feature_column.indicator_column?

In [11]:
def make_dataset(data_df, label_df, epochs =10, shuffle = True,
                batch_size = 32):
    # 这里将dataframe类型的dataset处理未dictionary的形式再做处理
    # Q：value的形式不可以么？
    dataset = tf.data.Dataset.from_tensor_slices(
        (dict(data_df), label_df))
    if shuffle:
        dataset = dataset.shuffle(10000)
    dataset = dataset.repeat(epochs).batch(batch_size)
    return dataset

In [12]:
output_dir = "baseline_model"
if not os.path.exists(output_dir):
    os.mkdir(output_dir)

# 定义estimator
baseline_estimator = tf.estimator.BaselineClassifier(
    model_dir = output_dir,
    n_classes = 2)

baseline_estimator.train(input_fn = lambda : make_dataset(
    train_df, y_train, epochs = 100))

W0829 21:17:56.251702 4623128000 deprecation.py:323] From /anaconda3/envs/tylab/lib/python3.6/site-packages/tensorflow/python/training/training_util.py:236: Variable.initialized_value (from tensorflow.python.ops.variables) is deprecated and will be removed in a future version.
Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.
W0829 21:17:56.436001 4623128000 deprecation.py:323] From /anaconda3/envs/tylab/lib/python3.6/site-packages/tensorflow_estimator/python/estimator/head/base_head.py:574: to_float (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.cast` instead.
W0829 21:17:56.547243 4623128000 deprecation.py:323] From /anaconda3/envs/tylab/lib/python3.6/site-packages/tensorflow/python/ops/nn_impl.py:182: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be r

FailedPreconditionError: GetNext() failed because the iterator has not been initialized. Ensure that you have run the initializer operation for this iterator before getting the next element.
	 [[node IteratorGetNext (defined at <ipython-input-12-d5468de5b31e>:10) ]]

Original stack trace for 'IteratorGetNext':
  File "/anaconda3/envs/tylab/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/anaconda3/envs/tylab/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/anaconda3/envs/tylab/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/anaconda3/envs/tylab/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/anaconda3/envs/tylab/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 505, in start
    self.io_loop.start()
  File "/anaconda3/envs/tylab/lib/python3.6/site-packages/tornado/platfo

In [13]:
1

1